In [69]:
import pickle
import re
import os
import glob
from slap_train_pipeline_v2 import SerializableResults

In [70]:
# Configuration
# Set specific run timestamp (e.g. "2025-12-14/17-21-03") or None to use most recent
TARGET_RUN = None 
BASE_OUTPUT_DIR = "/n/fs/jborz/projects/slap/outputs"

def get_run_dir(target_run=None, base_dir=BASE_OUTPUT_DIR):
    """Get the run directory, defaulting to most recent if not specified."""
    if target_run:
        # Check if full path or relative path
        if os.path.isabs(target_run):
            return target_run
        return os.path.join(base_dir, target_run)
    
    # Find most recent run
    # Pattern: base_dir/YYYY-MM-DD/HH-MM-SS
    date_dirs = sorted(glob.glob(os.path.join(base_dir, "*-*-*")))
    if not date_dirs:
        raise ValueError(f"No date directories found in {base_dir}")
        
    # Search latest date dir first
    for date_dir in reversed(date_dirs):
        time_dirs = sorted(glob.glob(os.path.join(date_dir, "*-*-*")))
        if time_dirs:
            return time_dirs[-1]
            
    raise ValueError(f"No run directories found in {base_dir}")

run_dir = get_run_dir(TARGET_RUN)
print(f"Using run directory: {run_dir}")

Using run directory: /n/fs/jborz/projects/slap/outputs/2025-12-14/20-06-34


In [71]:
with open(run_dir + "/results.pkl", "rb") as f:
    results: SerializableResults = pickle.load(f)

In [72]:
results.teleporter_locations

[((1, 0), (2, 1)), ((0, 1), (1, 2))]

In [73]:
def analyze_teleporters(results):
    if not results.teleporter_locations:
        print("No teleporter locations found.")
        return

    # 1) Create mapping from (col, row) -> node_id using training data
    data = results.final_training_data or results.training_data
    if not data:
        print("No training data found (final_training_data or training_data is missing).")
        return

    node_atoms = data.node_atoms
    coord_to_node = {}

    for node_id, atoms in node_atoms.items():
        col = -1
        row = -1

        for atom in atoms:
            atom_str = str(atom)

            # Match InColX and InRowY
            col_match = re.search(r"InCol(\d+)", atom_str)
            if col_match:
                col = int(col_match.group(1))

            row_match = re.search(r"InRow(\d+)", atom_str)
            if row_match:
                row = int(row_match.group(1))

        if col != -1 and row != -1:
            coord_to_node[(col, row)] = node_id

    # 2) Get shortcut sets
    if results.training_data:
        unique_shortcuts_set = set(results.training_data.unique_shortcuts)
    elif results.pruned_training_data:  # Fallback
        unique_shortcuts_set = set(results.pruned_training_data.unique_shortcuts)
    else:
        unique_shortcuts_set = set()

    pruned_set = set()
    if results.pruned_training_data:
        pruned_set = set(results.pruned_training_data.unique_shortcuts)

    final_shortcuts_set = set()
    if results.final_training_data:
        final_shortcuts_set = set(results.final_training_data.unique_shortcuts)

    # 3) Print table
    print(f"\nAnalyzing {len(results.teleporter_locations)} teleporters...")
    print(
        f"{'Teleporter (Coords)':<30} {'Node IDs':<15} "
        f"{'In Train?':<10} {'In Prune?':<10} {'In Final?'}"
    )
    print("-" * 80)

    for loc1, loc2 in results.teleporter_locations:
        loc1 = tuple(loc1)
        loc2 = tuple(loc2)

        id1 = coord_to_node.get(loc1)
        id2 = coord_to_node.get(loc2)

        coords_str = f"{loc1} <-> {loc2}"

        if id1 is None or id2 is None:
            print(f"{coords_str:<30} {'Not Found':<15} {'-':<10} {'-':<10} {'-'}")
            continue

        in_train = ((id1, id2) in unique_shortcuts_set) or ((id2, id1) in unique_shortcuts_set)
        in_prune = ((id1, id2) in pruned_set) or ((id2, id1) in pruned_set)
        in_final = ((id1, id2) in final_shortcuts_set) or ((id2, id1) in final_shortcuts_set)

        nodes_str = f"{id1} <-> {id2}"
        print(f"{coords_str:<30} {nodes_str:<15} {str(in_train):<10} {str(in_prune):<10} {str(in_final)}")


analyze_teleporters(results)



Analyzing 2 teleporters...
Teleporter (Coords)            Node IDs        In Train?  In Prune?  In Final?
--------------------------------------------------------------------------------
(1, 0) <-> (2, 1)              1 <-> 2         True       True       False
(0, 1) <-> (1, 2)              4 <-> 5         True       True       True


In [74]:
results.evaluation_results.episode_data[0]

{'initial_node': 0,
 'goal_nodes': [5],
 'optimal_path_nodes': [0, 1, 5],
 'optimal_path_edges': [{'source': 0,
   'target': 1,
   'is_shortcut': False,
   'cost': 4},
  {'source': 1, 'target': 5, 'is_shortcut': False, 'cost': inf}],
 'shortcuts_added': 7,
 'success': True,
 'true_steps': 13,
 'reward': 88.0}

In [75]:
def get_episodes_with_shortcuts(episode_data: list[dict]) -> list[int]:
    """
    Identifies and prints episodes where a shortcut was used in the optimal path.

    Args:
        episode_data: A list of dictionaries, where each dictionary
                      represents an episode and contains 'optimal_path_edges'.

    Returns:
        A list of episode indices where shortcuts were found.
    """
    episodes_with_shortcuts = []
    print("Checking for shortcuts in optimal paths:")
    for i, episode in enumerate(episode_data):
        optimal_path_edges = episode.get('optimal_path_edges', [])
        for edge in optimal_path_edges:
            if edge.get('is_shortcut', False):
                print(f"  Shortcut used in Episode {i}")
                episodes_with_shortcuts.append(i)
                break  # Only need to find one shortcut per episode
    
    if not episodes_with_shortcuts:
        print("  No shortcuts found in any optimal path.")

    return episodes_with_shortcuts

get_episodes_with_shortcuts(results.evaluation_results.episode_data)

Checking for shortcuts in optimal paths:
  No shortcuts found in any optimal path.


[]